In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


#**Load Kaggle Dataset**

In [7]:
!pip uninstall -y kaggle
!pip install -upgrade pip
!pip install kaggle==1.5.6
!kaggle -v

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u
     |████████████████████████████████| 61kB 2.3MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=e699256fe1b37a945bdb37589dba026ed532ef74cf7c571c39deaf417b40d60c
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.6/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/py

In [8]:
import os
os.environ['KAGGLE_USERNAME'] = "user_name" # username from the json file
os.environ['KAGGLE_KEY'] = "key" # key from the json file
!kaggle competitions download -c plant-pathology-2020-fgvc7
!unzip -q plant-pathology-2020-fgvc7.zip -d .

 98% 766M/779M [00:09<00:00, 119MB/s] 
100% 779M/779M [00:09<00:00, 87.0MB/s]


In [9]:
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd

from google.colab import files
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

root_path = './'
print(root_path + "train.csv")
train_df = pd.read_csv(root_path + "train.csv")
print(train_df.head())
# test_df =  pd.read_csv(root_path + "test.csv")
# sub_csv = pd.read_csv(root_path + "sample_submission.csv")


cuda:0
./train.csv
  image_id  healthy  multiple_diseases  rust  scab
0  Train_0        0                  0     0     1
1  Train_1        0                  1     0     0
2  Train_2        1                  0     0     0
3  Train_3        0                  0     1     0
4  Train_4        1                  0     0     0


In [0]:
class leafDataset(Dataset):

    def __init__(self, root_path, dataFrame, transform = None):
        # self.file_path = file_path
        self.classes = list(dataFrame)[1:]
        self.root_path = root_path
        self.transform = transform
        # Some preprocessing
   
        self.df = dataFrame

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = self.root_path + 'images/' + str(self.df.iloc[idx,0]) + '.jpg'
        X = Image.open(img_name)
        y = np.asarray(self.df.iloc[idx, 1:], dtype = np.float32)
            
        sample = {"image" : X , "conditions" : y}
        
        if self.transform:
            sample['image'] = self.transform(sample['image'])

        
        return sample

    def __len__(self):
        return len(self.df)



In [0]:
from torchvision import transforms
transform = transforms.Compose(
    [
     transforms.Resize((64,96)),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.ToTensor(),    # range [0, 255] -> [0.0,1.0]
     transforms.Normalize((0.5, ), (0.5, ))])  
full_set = leafDataset(root_path,train_df,transform) 

# **Split Dataset**

In [0]:
from torch.utils.data import random_split

train_size = int(0.8 * len(full_set))
train_set, val_set = random_split(full_set, [train_size, len(full_set)-train_size ])


# **Model Definition**

In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

model = nn.Sequential(
          nn.Conv2d(3,16,3),
          nn.ReLU(),
          nn.Conv2d(16,16,3),
          nn.ReLU(),
          nn.BatchNorm2d(16),
          nn.Conv2d(16,16,3),
          nn.ReLU(),
          nn.MaxPool2d(2, 2),
          nn.Conv2d(16,32,3),
          nn.ReLU(),
          nn.Conv2d(32,32,3),
          nn.BatchNorm2d(32),
          nn.ReLU(),
          nn.MaxPool2d(2, 2),
          nn.Conv2d(32,64,3),
          nn.ReLU(),
          nn.Conv2d(64,64,3),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          nn.MaxPool2d(2, 2),
          nn.Conv2d(64,128,3),
          nn.ReLU(),
          nn.MaxPool2d(2, 2),
          nn.Flatten(),
          nn.Linear(384,192),
          nn.BatchNorm1d(192),
          nn.ReLU(),
          nn.Linear(192,48),
          nn.ReLU(),
          nn.Linear(48,4),
          nn.Sigmoid()
        )


In [14]:
from torchsummary import summary
# model = Net()
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)
model.apply(init_weights)
model = model.to(device)
summary(model, input_size=full_set[0]['image'].shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 62, 94]             448
              ReLU-2           [-1, 16, 62, 94]               0
            Conv2d-3           [-1, 16, 60, 92]           2,320
              ReLU-4           [-1, 16, 60, 92]               0
       BatchNorm2d-5           [-1, 16, 60, 92]              32
            Conv2d-6           [-1, 16, 58, 90]           2,320
              ReLU-7           [-1, 16, 58, 90]               0
         MaxPool2d-8           [-1, 16, 29, 45]               0
            Conv2d-9           [-1, 32, 27, 43]           4,640
             ReLU-10           [-1, 32, 27, 43]               0
           Conv2d-11           [-1, 32, 25, 41]           9,248
      BatchNorm2d-12           [-1, 32, 25, 41]              64
             ReLU-13           [-1, 32, 25, 41]               0
        MaxPool2d-14           [-1, 32,

# **Training**

In [0]:
def save_checkpoint(save_path, model, optimizer, val_loss):
    if save_path==None:
        return
    save_path = save_path 
    state_dict = {'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'val_loss': val_loss}

    torch.save(state_dict, save_path)

    print(f'Model saved to ==> {save_path}')

def load_checkpoint(model, optimizer):
    save_path = f'final_model.pt'
    state_dict = torch.load(save_path)
    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    val_loss = state_dict['val_loss']
    print(f'Model loaded from <== {save_path}')
    
    return val_loss



def TRAIN(net, train_loader, valid_loader,  num_epochs, eval_every, total_step, criterion, optimizer, val_loss, device, save_name):
    train_loss_his = []
    val_loss_his = []
    running_loss = 0.0
    # running_corrects = 0
    # running_num = 0
    global_step = 0
    if val_loss == None:
        best_val_loss = float("Inf")  
    else: 
        best_val_loss = val_loss
    

    for epoch in range(num_epochs):  # loop over the dataset multiple times

        for i, samples in enumerate(train_loader):
            inputs = samples['image']
            labels = samples['conditions']
            # print(inputs.shape)
            net.train()                             # set to trainning mode
            inputs = inputs.to(device)              
            labels = labels.to(device)
            '''Training of the model'''
            # Forward pass
            outputs = net(inputs)
            # _, preds = torch.max(outputs.data, 1)   # get the predicted output val,index


            loss = criterion(outputs, labels)       # calc the loss
            
            # Backward and optimize
            optimizer.zero_grad()                   
            loss.backward()                         # calc grad
            optimizer.step()                        # refreash the weight
            global_step += 1

            # print("here1")
            running_loss += loss.item()
            # running_corrects += torch.sum(preds == labels.data)
            # running_num += len(labels)
            
            '''Evaluating the model every x steps'''
            if global_step % eval_every == 0:
                with torch.no_grad():
                    net.eval()
                    val_running_loss = 0
                    
                    # val_running_corrects = 0
                    for samples in valid_loader:
                            val_inputs = samples['image'].to(device)
                            val_labels = samples['conditions']
                            val_outputs = net(val_inputs)
                            val_loss = criterion(val_outputs.to(device), val_labels.to(device)).to(device)
                            # _, preds = torch.max(val_outputs.data, 1)
                            val_running_loss += val_loss.item()
                            # val_running_corrects += torch.sum(preds == val_labels.data)

                    average_train_loss = running_loss / eval_every
                    average_val_loss = val_running_loss / len(valid_loader)
                    # average_train_acc = running_corrects / float(running_num)
                    # average_val_acc = val_running_corrects / float(len(valid_loader))
                    train_loss_his.append(average_train_loss)
                    val_loss_his.append(average_val_loss)
                    print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}'.format(epoch+1, num_epochs, global_step, total_step, average_train_loss,average_val_loss))

                    running_loss = 0.0
                    running_num = 0
                    # running_corrects = 0
                    
                    if average_val_loss < best_val_loss:
                        best_val_loss = average_val_loss
                        save_checkpoint(save_name, net, optimizer, best_val_loss)
                    
                    

    print('Finished Training')
    return (train_loss_his,val_loss_his)

In [0]:
num_epochs = 10
eval_every = 100
train_loader = torch.utils.data.DataLoader(train_set, batch_size= len(train_set)//100,
                                          shuffle=True,num_workers=8)
valid_loader = torch.utils.data.DataLoader(val_set, batch_size= len(val_set)//25,
                                          shuffle=True,num_workers=8)

total_step = len(train_loader)*num_epochs
best_val_loss = None #0.1028
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters() , lr = 0.003 ,weight_decay = 1e-5)
save_path = f'new_model.pt'


his = TRAIN(model, train_loader, valid_loader, num_epochs, eval_every, total_step, criterion, optimizer, best_val_loss, device, save_path)

#**Print Result**

In [0]:
def plot_his(his):
  import matplotlib.pyplot as plt
  y1 = his[0]
  y2 = his[1]
  x = range(len(his[0]))

  print(best_val_loss)
  plt.plot(x,y1,label = "train loss")
  plt.plot(x,y2,label = "val loss")


  plt.title("Loss Graph")
  plt.legend()  
  plt.show() 

In [0]:
plot_his(his)

In [0]:
def eval(net , test_loader, thersholds):
  y_test = []
  y_pred = {}
  for j in thersholds:
    y_pred[str(j)] = []
  
  
  with torch.no_grad():
    for i in range(len(test_loader)):
      net.eval()
      samples = iter(test_loader).next()
      outputs = net(samples['image'].to(device))
      labels = samples['conditions']
      labels = labels.cpu()
      for thershold in thersholds:
        predicted = torch.where(outputs > thershold, torch.Tensor([1]).to(device) , torch.Tensor([0]).to(device))
        predicted = predicted.cpu()
        y_pred[str(thershold)].append(predicted)
      y_test.append(labels)
  return y_test, y_pred



In [0]:
valid_loader = torch.utils.data.DataLoader(val_set, batch_size= len(val_set)//25,
                                          shuffle=True,num_workers=8)
thersholds = [0.5] # np.arange(0.1,1,0.1)
y_test, y_pred = eval(model,valid_loader, thersholds)


In [0]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
try:
  y_test = torch.stack(y_test)
except:
  pass
for i in thersholds:
  try:
    y_pred[str(i)] = torch.stack(y_pred[str(i)])
  except:
    pass
  class_re = classification_report(y_test.reshape(-1,4) , y_pred[str(i)].reshape(-1,4), zero_division=0)
  acc = accuracy_score(y_test.reshape(-1,4) , y_pred[str(i)].reshape(-1,4))
  print(f'Thershold {round(i*10)/10} (with accuacry {acc}) report:\n {class_re}')

Thershold 0.5 (with accuacry 0.9365079365079365) report:
               precision    recall  f1-score   support

           0       0.97      0.95      0.96       122
           1       0.50      0.08      0.13        13
           2       0.96      0.98      0.97       112
           3       0.93      0.98      0.96       131

   micro avg       0.95      0.94      0.94       378
   macro avg       0.84      0.75      0.75       378
weighted avg       0.94      0.94      0.93       378
 samples avg       0.94      0.94      0.94       378



# **Load Pre-trained Best Model**

In [15]:
def load_checkpoint(model, optimizer):
    save_path = f'final_model.pt'
    state_dict = torch.load(save_path)
    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    val_loss = state_dict['val_loss']
    print(f'Model loaded from <== {save_path}')
    
    return val_loss
optimizer = optim.Adam(model.parameters() , lr = 0.001 ,weight_decay = 1e-5)
best_val_loss = load_checkpoint(model, optimizer)

Model loaded from <== final_model.pt


In [16]:
print(best_val_loss)

0.10283864729313387
